Import Libraries

In [50]:
import pandas as pd
import time
import os
from datetime import datetime
import numpy as np

Data Preparation

In [51]:
root_dir = 'source/'
files = os.listdir(root_dir + 'used')
all_dfs = []
for file in files:
    if '.csv' in file:
        df = pd.read_csv(root_dir + 'used/' + file)
        df.iloc[0,-1] = 'Team'
        new_columns = [f"{col}_{df.iloc[0][col]}" for col in df.columns]
        df = df[1:]
        df.columns = new_columns
        all_dfs.append(df)

df = pd.concat(all_dfs,ignore_index = True)
df = df.dropna()
df.to_csv(root_dir + 'summary.csv', index = False)

Loading the Summary table

In [52]:
df = pd.read_csv(root_dir + 'summary.csv')

In [53]:
df_bruno = df[df['Unnamed: 0_level_0_Player'] == 'Bruno Fernandes']
df_bruno.sort_values(by=['Unnamed: 4_level_0_Age'])

,Unnamed: 0_level_0_Player,Unnamed: 1_level_0_#,Unnamed: 2_level_0_Nation,Unnamed: 3_level_0_Pos,Unnamed: 4_level_0_Age,Unnamed: 5_level_0_Min,Performance_Gls,Performance.1_Ast,Performance.2_PK,Performance.3_PKatt,...,SCA.1_GCA,Passes_Cmp,Passes.1_Att,Passes.2_Cmp%,Passes.3_PrgP,Carries_Carries,Carries.1_PrgC,Take-Ons_Att,Take-Ons.1_Succ,Team_Team
59590,Bruno Fernandes,18.0,pt POR,"AM,CM",25-146,90,0,0,0,0,...,0,70,90,77.8,14,63,1,1,1,Manchester United
24736,Bruno Fernandes,18.0,pt POR,AM,25-162,89,0,1,0,0,...,1,33,43,76.7,5,22,1,0,0,Manchester United
59085,Bruno Fernandes,18.0,pt POR,AM,25-168,90,1,1,1,1,...,2,49,70,70.0,8,43,7,3,3,Manchester United
33747,Bruno Fernandes,18.0,pt POR,AM,25-175,90,1,0,0,0,...,0,44,70,62.9,5,43,5,2,2,Manchester United
58088,Bruno Fernandes,18.0,pt POR,"AM,FW",25-182,87,0,1,0,0,...,2,20,26,76.9,1,15,1,1,0,Manchester United
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29375,Bruno Fernandes,8.0,pt POR,CM,31-083,90,0,2,0,0,...,2,37,49,75.5,5,29,0,1,0,Manchester United
59308,Bruno Fernandes,8.0,pt POR,"CM,AM",31-087,90,0,0,0,0,...,0,62,81,76.5,9,35,2,1,0,Manchester United
88814,Bruno Fernandes,8.0,pt POR,CM,31-091,90,2,1,1,1,...,1,54,70,77.1,8,33,2,1,0,Manchester United
55726,Bruno Fernandes,8.0,pt POR,CM,31-098,90,1,1,0,0,...,1,46,61,75.4,8,38,2,1,0,Manchester United


In [54]:
# df['Passes.1_Att']
# df['Unnamed: 5_level_0_Min']
(df['Passes.1_Att']/df['Unnamed: 5_level_0_Min']).sum()
# 15/78

48476.1748654458

# Function Declaration
Create the features from historical stat, like passing, tackling. Then use it to train the model.

In [55]:
root_dir = 'source/'
files = os.listdir(root_dir + 'used')
files[0]

def change_columns(team_df):
    team_df = pd.read_csv(team_df)
    team_df.iloc[0,-1] = 'Team'
    new_columns = [f"{col}_{team_df.iloc[0][col]}" for col in team_df.columns]
    team_df = team_df[1:]
    team_df.columns = new_columns
    team_df = team_df.dropna()
    return team_df

def extract_team_features(team_df, is_home = True):
    features = {}


    features['avg_minutes'] = team_df['Unnamed: 5_level_0_Min'].astype('float32').mean()
    # Shooting
    features['total_shots'] = team_df['Performance.4_Sh'].astype('float32').sum()
    features['shots_on_target'] = team_df['Performance.5_SoT'].astype('float32').sum()
    features['xG'] = team_df['Expected_xG'].astype('float32').sum()
    features['xAG'] = team_df['Expected.2_xAG'].astype('float32').sum()
    # Passing
    features['key_passes'] = team_df['SCA_SCA'].astype('float32').sum() # Shot creating action
    features['pass_completion'] = team_df['Passes_Cmp'].astype('float32').sum()/team_df['Passes.1_Att'].astype('float32').sum()*100

    # Defensive
    features['tackles'] = team_df['Performance.9_Tkl'].astype('float32').sum()
    features['interception'] = team_df['Performance.10_Int'].astype('float32').sum()
    features['blocks'] = team_df['Performance.11_Blocks'].astype('float32').sum()

    # Cards
    features['yellow_cards'] = team_df['Performance.6_CrdY'].astype('float32').sum()
    features['red_cards'] = team_df['Performance.7_CrdR'].astype('float32').sum()
    # Position-specific
    positions = team_df['Unnamed: 3_level_0_Pos'].astype(str)

    # Attackers (FW, LW, RW, ST)
    attackers = team_df[positions.str.contains('FW|LW|RW|ST|AM')]

    if len(attackers) > 0:
        features['attackers_xG'] = attackers['Expected_xG'].astype('float32').sum()
        features['attackers_shots'] = attackers['Performance.4_Sh'].astype('float32').sum()
        # features['attackers_xG_minute'] = (attackers['Expected_xG'].astype('float32')/attackers['Unnamed: 5_level_0_Min'].astype('float32')).sum()
        # features['attackers_shots_minute'] = (attackers['Performance.4_Sh'].astype('float32')/attackers['Unnamed: 5_level_0_Min'].astype('float32')).sum()

    midfielders = team_df[positions.str.contains('CM|DM|LM|RM|AM')]
    
    if len(midfielders) > 0:
        features['midfielders_passes'] = midfielders['Passes_Cmp'].astype('float32').sum()/midfielders['Passes.1_Att'].astype('float32').sum()*100
        # features['midfielders_passes_minute'] = (midfielders['Passes_Cmp'].astype('float32')/attackers['Unnamed: 5_level_0_Min'].astype('float32')).sum()
    defenders = team_df[positions.str.contains('CB|RB|LB|WB|DF')]

    if len(defenders) > 0:
        features['defenders_tackles'] = defenders['Performance.9_Tkl'].astype('float32').sum()
        features['defenders_blocks'] = defenders['Performance.11_Blocks'].astype('float32').sum()
        # features['defenders_tackles_minute'] = (defenders['Performance.9_Tkl'].astype('float32')/attackers['Unnamed: 5_level_0_Min'].astype('float32')).sum()
        # features['defenders_blocks_minute'] = (defenders['Performance.11_Blocks'].astype('float32')/attackers['Unnamed: 5_level_0_Min'].astype('float32')).sum()

    return features


def process_match_file(filepath):
    df = change_columns(filepath)
    home_data = df[df['Team_Team'] == df.iloc[0,-1]]
    away_data = df[df['Team_Team'] == df.iloc[-1,-1]]
    home_features = extract_team_features(home_data, is_home = True)
    away_features = extract_team_features(away_data, is_home = False)

    match_features = {}
    for key, value in home_features.items():
        match_features[f'home_{key}'] = value

    for key, value in away_features.items():
        match_features[f'away_{key}'] = value

    for key in home_features.keys():
        if key in away_features:
            match_features[f'diff_{key}'] = home_features[key] - away_features[key]
            match_features[f'ratio_{key}'] = home_features[key]/(away_features[key] + 0.00001) # avoid dividing by zero

    return match_features            

# Create Training Dataset

In [56]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import xgboost as xgb

In [57]:
# creating results dict
results_dict = {}
for file in files:
    df = change_columns(root_dir+'/used/'+file)
    match_id = os.path.basename(file).replace('.csv','')
    home_goals = df[df['Team_Team'] == df.iloc[0,-1]]['Performance_Gls'].astype('float32').sum()
    away_goals = df[df['Team_Team'] == df.iloc[-1,-1]]['Performance_Gls'].astype('float32').sum()
    if home_goals > away_goals:
        results_dict[match_id] = 'H'
    if home_goals < away_goals:
        results_dict[match_id] = 'A'
    if home_goals == away_goals:
        results_dict[match_id] = 'D'
    

In [58]:
def build_training_dataset(match_files, results_dict):
    all_features = []
    all_labels = []
    
    for match_file in match_files:
        match_id = os.path.basename(match_file).replace('.csv','')
        if match_id in results_dict:
            features = process_match_file(root_dir+'used/'+match_file)
            result = results_dict[match_id]
            label_map = {'H':0, 'D':1,'A':2}
            if result in label_map:
                all_features.append(features)
                all_labels.append(label_map[result])
    features_df = pd.DataFrame(all_features)
    labels_array = np.array(all_labels)

    return features_df, labels_array


In [59]:
match_files = os.listdir(root_dir + 'used')
X,y = build_training_dataset(match_files, results_dict)

In [60]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=100)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [61]:
X_train_scaled

array([[ 1.04230761,  1.29096023, -0.99112903, ..., -0.03966863,
        -0.40854308, -0.10375939],
       [ 2.08405104,  1.81845827,  0.99989454, ..., -0.03978359,
        -1.16607486, -0.10376336],
       [ 1.04230761, -1.17069731, -0.59292432, ..., -0.03973531,
        -1.16607486, -0.10376336],
       ...,
       [ 0.05899938,  3.22511972,  3.78732755, ..., -0.03979508,
        -1.16607486, -0.10376543],
       [ 0.05899938, -0.11570122, -0.99112903, ..., -0.03978071,
        -1.16607486, -0.10376336],
       [-0.79320137,  2.521789  ,  1.39809926, ..., -0.03977209,
         0.0964781 , -0.10375614]])

In [62]:
import joblib
joblib.dump(scaler, 'scaler/scaler.pkl')
print('Successfully save scaler')

Successfully save scaler


# Model Training

In [63]:
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error, r2_score

In [64]:
xgb_model = xgb.XGBClassifier(
    n_estimators=100,
    learning_rate=0.1,
    max_depth=5,
    objective='multi:softprob',  # For multi-class
    num_class=3,
    random_state=42
)

xgb_model.fit(X_train_scaled,y_train)

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.1, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None, num_class=3,
              num_parallel_tree=None, ...)

In [65]:
y_pred = xgb_model.predict(X_test_scaled)
mse = mean_squared_error(y_test, y_pred)
print("MSE:", mse)

# Calculating R squared using sklearn
r2 = r2_score(y_test, y_pred)
print("R squared:", r2)

MSE: 0.60062893081761
R squared: 0.16335097645555763


In [66]:
X_test_scaled

array([[ 0.05899938, -0.81903195, -0.1947196 , ..., -0.0397909 ,
        -1.16607486, -0.10376336],
       [ 0.05899938,  0.06013146, -0.59292432, ..., -0.03977497,
         0.60149929, -0.10375094],
       [-1.53887685,  1.81845827,  1.79630397, ..., -0.03978562,
        -2.17611723, -0.10376514],
       ...,
       [ 0.05899938, -0.81903195,  0.20348511, ..., -0.03977497,
        -1.16607486, -0.10376427],
       [-1.53887685, -0.46736658, -0.1947196 , ..., -0.03976881,
         0.3489887 , -0.10375289],
       [ 0.05899938, -0.2915339 , -0.59292432, ..., -0.03976443,
        -1.67109605, -0.10376914]])

In [67]:
xgb_model.save_model('model/model.json')